In [1]:
import os
import certifi
from river import datasets
from confluent_kafka import Producer,Consumer
import certifi
import time
import json

In [2]:
os.environ['FEATURE_TOPIC_NAME']='malicious_url_events'
os.environ['INFERENCE_GROUP_ID']='test-grp-100'
os.environ['PREDICTION_TOPIC_PREFIX']='malicious_url_predictions'
os.environ['PREDICTION_TOPIC_SUFFIX']='_8'
prediction_topic_prefix = os.environ['PREDICTION_TOPIC_PREFIX']
prediction_topic_suffix=os.environ['PREDICTION_TOPIC_SUFFIX']
PREDICTION_TOPIC=f'{prediction_topic_prefix}{prediction_topic_suffix}'

KAFKA_BOOTSTRAP_SERVERS = os.environ.get('kafka_bootstrap_servers')
KAFKA_USER_NAME = os.environ.get('kafka_username')
KAFKA_PASSWORD = os.environ.get('kafka_password')

In [3]:
#import src.online_learning_model

In [4]:
from river import metrics
auc = metrics.ROCAUC()
f1 = metrics.F1()
recall = metrics.MicroRecall()
predictions_consumer_conf = {'bootstrap.servers': KAFKA_BOOTSTRAP_SERVERS,
                     'sasl.username': KAFKA_USER_NAME,
                     'sasl.password': KAFKA_PASSWORD,
                     'sasl.mechanism': 'PLAIN',
                     'security.protocol': 'SASL_SSL',
                     'ssl.ca.location': certifi.where(),
                     'group.id': 'prediction-grp-2',
                     'enable.auto.commit': True,
                     'auto.offset.reset': 'earliest'}
predictions_consumer = Consumer(predictions_consumer_conf)    
predictions_consumer.subscribe([PREDICTION_TOPIC])

In [ ]:

durs = []
mem_usage = []
cnt = 0
while(cnt<10000):
    msg = predictions_consumer.poll(timeout=0.1)
    
    if msg is None: continue
    if msg.error():
        if msg.error().code() == KafkaError._PARTITION_EOF:
                sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                         (msg.topic(), msg.partition(), msg.offset()))
    else:            
        message = json.loads(msg.value().decode("utf-8"))
        print(message)
        if(cnt%1000==0):
            print(f'Processed{cnt}')
            
        cnt = cnt + 1
        score = message['score']
        y = message['y']
        durs.append(message['duration'])
        mem_usage.append(message['mem_usage'])
        auc = auc.update(y, score)
        f1 = f1.update(y, score)
        recall = recall.update(y, score)

{'y': False, 'score': False, 'duration': 9823.633985042572, 'mem_usage': 233479167}
Processed0
{'y': True, 'score': True, 'duration': 9831.623094558716, 'mem_usage': 233784225}
{'y': True, 'score': True, 'duration': 9839.522679805756, 'mem_usage': 233792369}
{'y': True, 'score': False, 'duration': 9847.38647532463, 'mem_usage': 233844005}
{'y': False, 'score': False, 'duration': 9855.312207698822, 'mem_usage': 234089077}
{'y': True, 'score': True, 'duration': 9861.02959561348, 'mem_usage': 234479215}
{'y': True, 'score': True, 'duration': 9836.151253700256, 'mem_usage': 233792369}
{'y': True, 'score': True, 'duration': 9842.886917114258, 'mem_usage': 233804704}
{'y': True, 'score': True, 'duration': 9850.772818803787, 'mem_usage': 233950256}
{'y': True, 'score': True, 'duration': 9859.891321182251, 'mem_usage': 234347083}
{'y': False, 'score': False, 'duration': 9821.35885977745, 'mem_usage': 233368886}
{'y': False, 'score': False, 'duration': 9827.055046081543, 'mem_usage': 233628365}

In [ ]:
import statistics
mean = statistics.mean(durs)
median = statistics.median(durs)
max_dur = max(durs)
min_dur = min(durs)
avg_dur = min(durs)
total_records = len(durs)
memory_usage = statistics.mean(mem_usage)
print(max_dur)
print(min_dur)
print(avg_dur)
print(auc)
print(f1)
print(recall)
print(memory_usage)

In [ ]:
print(len(durs))